In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone as PineconeClient
from pinecone import ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from sentence_transformers import SentenceTransformer
import torch
import os
import time

In [ ]:
#Extract env variables
from dotenv import load_dotenv
load_dotenv()

In [ ]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

extracted_data = load_pdf("../data/")

In [ ]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))
text_chunks

In [ ]:
pc = PineconeClient(api_key=os.environ.get('PINECONE_API_KEY'),
                    proxy_url=os.environ.get('PROXY_URL'))

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)

In [ ]:
index_name="genaichatbot"

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=384,  # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings
embeddings = download_hugging_face_embeddings()

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2').to(device)
sentences = [t.page_content for t in text_chunks]

In [ ]:
embeddings_sentences =  model.encode(sentences)  
id = 0
vectors = []
for d, e in zip(text_chunks, embeddings_sentences):
    id = id+1
    vectors.append({
        "id": str(id),
        "values": e,
        "metadata": {'page_content': d.page_content}
    })

index.upsert(
    vectors=vectors,
    namespace="nschatbot"
)

In [ ]:
query = "Tell me about Gabriel Garcia Marquez"

query_embedding = model.encode(query).tolist()
print(query_embedding)

results = index.query(
    namespace="nschatbot",
    vector=query_embedding,
    top_k=3,
    include_values=False,
    include_metadata=True
)
#print(results)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings) 

qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])